# Stage 1: Automatically compute derivatives

In [3]:
import numpy as np

## Step 1: Variables as boxes

将变量比作"箱子"
- "箱子"和数据是不同的东两
- "箱子"里可以存放数据(=赋值)
- 朝"箱子"里看一看就能知道数据是什么(=引用)

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [4]:
data = np.array(1.0)
x = Variable(data)
print(x.data) # 1.0

x.data = np.array(2.0)
print(x.data) # 2.0

1.0
2.0


## Step 2: Function to create a variable

函数是定义一个变量与另一个变量之间的对应关系的规则
- 在 Function 类中实现的方法，其输入应为 Variable 实例，输出应为Variable 实例
- Variable 实例的实际数据存在于实例变量 data 中

In [7]:
# 函数的基类
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

In [8]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [9]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y)) # <class '__main__.Variable'>
print(y.data) # 100

<class '__main__.Variable'>
100


## Step 3: Connecting Functions

函数连续调用，可以视作一个大函数(复合函数)  
计算图：  
x -> A -> a -> B -> b -> C -> y

In [10]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [11]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data) # 1.648721270700128

1.648721270700128


## Step 4: Numerical Differentiation

$$ \frac{df}{dx} = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h} $$

将h设为一个很小的数，计算导数的近似值(数值微分)  
中心差分误差更小：
$$ \frac{df}{dx} = \lim_{h \to 0} \frac{f(x+h) - f(x-h)}{2h} $$

但数值微分结果容易包含误差，更严重的是计算量大

In [12]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [13]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy) # 4.000000000004

4.000000000004


In [14]:
# 复合函数
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy) # 3.2974426293330694

3.2974426293330694


## Step 5: Theory of Backpropagation

## Step 6: Backpropagation by hand

## Step 7: Automation of backpropagation

## Step 8: From Recusion to Loop

## Step 9: Making Funcions More Useful

## Step 10: Perfrom the test